# Overview

# Business Understanding

# Data Understanding

## Data Preparation

In [1]:
import kagglehub
import pandas as pd
import os


In [2]:
# Download latest dataset version
path = kagglehub.dataset_download("thedevastator/booking-com-hotel-reviews")

# Identify the path for the dataset
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/booking-com-hotel-reviews


In [5]:
# Downloading the latest data
#base_dataset_path = "/kaggle/input/thedevastator/booking-com-hotel-reviews"

In [15]:
# The path where the data is in the environment
base_dataset_directory = "/kaggle/input/booking-com-hotel-reviews"
csv_file_name = "booking_reviews copy.csv"
full_csv_path = os.path.join(base_dataset_directory, csv_file_name)
df = pd.read_csv(full_csv_path)


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26675 entries, 0 to 26674
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            26675 non-null  int64  
 1   review_title     26674 non-null  object 
 2   reviewed_at      26570 non-null  object 
 3   reviewed_by      26570 non-null  object 
 4   images           938 non-null    object 
 5   crawled_at       26386 non-null  object 
 6   url              26386 non-null  object 
 7   hotel_name       26386 non-null  object 
 8   hotel_url        26386 non-null  object 
 9   avg_rating       26386 non-null  float64
 10  nationality      26370 non-null  object 
 11  rating           26386 non-null  float64
 12  review_text      26386 non-null  object 
 13  raw_review_text  26202 non-null  object 
 14  tags             26202 non-null  object 
 15  meta             26202 non-null  object 
dtypes: float64(2), int64(1), object(13)
memory usage: 3.3+ MB


# Exploratory Data Understanding